In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import shap

import pe_utils as helper

import matplotlib as mpl
import matplotlib.pyplot as plt


import seaborn as sns
font_size = 10
rc={'font.size': font_size, 'axes.labelsize': font_size, 'figure.dpi':400, 'axes.linewidth':0.5,
    'axes.titlesize': font_size, 'xtick.labelsize': font_size, 'ytick.labelsize': font_size} # 'figure.figsize':(11.7/1.5,8.27/1.5)

sns.set(style='ticks',rc=rc) #talk, ticks, paper
sns.set_context("paper")

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
plt.rcParams['axes.unicode_minus']=False # negative minus sign
plt.rcParams['axes.grid'] = False

In [2]:
import sklearn
sklearn.__version__

'1.0.2'

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')

%matplotlib inline

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.5 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.0.2 statsmodels==0.13.2 python-igraph==0.10.2 pynndescent==0.5.7


In [4]:
# dir_root=r'E:\pbmc_pe\data'
adata_fname = r'E:\pbmc_pe\data\Matrix\win_sc_data.h5ad'
adata = sc.read_h5ad(adata_fname)

In [5]:
adata.raw = adata

In [6]:
adata

AnnData object with n_obs × n_vars = 28774 × 17809
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'percent.ercc', 'percent.rp', 'percent.mt', 'percent.ncRNA', 'percent.LOC', 'percent.HB', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.3', 'seurat_clusters', 'celltype_l1', 'celltype_l2'

# preprocessing

In [7]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# assume data has been filtered
# adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# filter gene
gene_excl = 'HBB,HBA1,HBA2,BBPB,HGHA1,IGHG1,HGHM,IGKC,IGLC3,PDE4B,FTH1P2,HLA-J,FAUP1,PTMAP2,FTH1P8'.split(',')
adata = adata[:, [x for x in adata.var_names if x not in gene_excl]]

# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.highly_variable_genes(adata,  n_top_genes=2000)
adata = adata[:, adata.var.highly_variable]


filtered out 41 cells that have less than 200 genes expressed
filtered out 241 genes that are detected in less than 3 cells


Received a view of an AnnData. Making a copy.


normalizing counts per cell
    finished (0:00:00)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)


Trying to modify attribute `._uns` of view, initializing view as actual.


--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


# combine cell type

In [8]:
"""
Total B: Naive B 1; Naive B 2; Naive B 3; Unswitched Memory B; Switched Memory B 1; Switched Memory B 2; Plasma B
Total T and NK: XCL+ NK; CD16+ NK; CD160+ NK; CD8+ NK-like; CD8+ CTL; CD8+ Tn 1; CD8+ Tn 2; MAIT; CD4+ Tcm; CD4+ Tm; CD4+ Tn 1; CD4+ Tn 2; Treg; Cycling T; γδT
CD4+ Tn 2
CD8+ Tn 2
Mono: Mono;IFN+ non-classical Mono;non-classical Mono;Classical Mono;IFN+ non-classical Mono
Treg
"""

'\nTotal B: Naive B 1; Naive B 2; Naive B 3; Unswitched Memory B; Switched Memory B 1; Switched Memory B 2; Plasma B\nTotal T and NK: XCL+ NK; CD16+ NK; CD160+ NK; CD8+ NK-like; CD8+ CTL; CD8+ Tn 1; CD8+ Tn 2; MAIT; CD4+ Tcm; CD4+ Tm; CD4+ Tn 1; CD4+ Tn 2; Treg; Cycling T; γδT\nCD4+ Tn 2\nCD8+ Tn 2\nMono: Mono;IFN+ non-classical Mono;non-classical Mono;Classical Mono;IFN+ non-classical Mono\nTreg\n'

In [9]:
total_b_str = 'Naive B 1; Naive B 2; Naive B 3; Unswitched Memory B; Switched Memory B 1; Switched Memory B 2; Plasma B'
total_t_nk_str = 'XCL+ NK; CD16+ NK; CD160+ NK; CD8+ NK-like; CD8+ CTL; CD8+ Tn 1; CD8+ Tn 2; MAIT; CD4+ Tcm; CD4+ Tm; CD4+ Tn 1; CD4+ Tn 2; Treg; Cycling T; γδT'
mono_str = 'Mono;Non-classical Mono;Classical Mono;IFN+ Non-classical Mono'

total_b = [x.strip() for x in total_b_str.split(';')]
total_t_nk = [x.strip() for x in total_t_nk_str.split(';')]
mono = [x.strip() for x in mono_str.split(';')]

In [10]:
celltype_l3 = adata.obs['celltype_l2'].copy().to_frame()

celltype_l3['celltype_l3']=celltype_l3['celltype_l2'].apply(lambda x: 'Total B' if x in total_b else x)
celltype_l3['celltype_l3']=celltype_l3['celltype_l3'].apply(lambda x: 'Total T and NK' if x in total_t_nk else x)
celltype_l3['celltype_l3']=celltype_l3['celltype_l3'].apply(lambda x: 'Mono' if x in mono else x)

In [11]:
# from collections import Counter
# print('n_cells'+'--'*10)
# print(Counter(adata.obs['group'].values))

# cell_type1=Counter(adata.obs['celltype_l1'].values)
# print('n_cells per type1'+'--'*10)
# print(cell_type1.most_common())

# cell_type2=Counter(adata.obs['celltype_l2'].values)
# print('n_cells per type2'+'--'*10)
# print(cell_type2.most_common())

# # patients number
# print('n_patients'+'--'*10)
# print(np.unique(adata.obs['orig.ident'].values))
# # 23(NP:15, PE:8)

In [12]:
Total_B = celltype_l3[celltype_l3['celltype_l3']=='Total B']
Total_T_NK = celltype_l3[celltype_l3['celltype_l3']=='Total T and NK']
Total_Mono = celltype_l3[celltype_l3['celltype_l3']=='Mono']
cd4_tn2 = celltype_l3[celltype_l3['celltype_l2']=='CD4+ Tn 2']
cd8_tn2 = celltype_l3[celltype_l3['celltype_l2']=='CD8+ Tn 2']
Treg = celltype_l3[celltype_l3['celltype_l2']=='Treg']


In [13]:

final_cell_types={'Total_B':Total_B, 'Total_T_NK':Total_T_NK, 'Total_Mono':Total_Mono, 'cd4_tn2':cd4_tn2, 'cd8_tn2':cd8_tn2, 'Treg':Treg}


In [14]:
obs_celltype=pd.DataFrame(index=adata.obs_names)
obs_celltype['Total_B']=adata.obs_names.isin(Total_B.index)
obs_celltype['Total_T_NK']=adata.obs_names.isin(Total_T_NK.index)
obs_celltype['Total_Mono']=adata.obs_names.isin(Total_Mono.index)
obs_celltype['cd4_tn2']=adata.obs_names.isin(cd4_tn2.index)
obs_celltype['cd8_tn2']=adata.obs_names.isin(cd8_tn2.index)
obs_celltype['Treg']=adata.obs_names.isin(Treg.index)

adata.obs[obs_celltype.columns] = obs_celltype

Trying to modify attribute `.obs` of view, initializing view as actual.


In [15]:
adata

AnnData object with n_obs × n_vars = 28733 × 2000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'percent.ercc', 'percent.rp', 'percent.mt', 'percent.ncRNA', 'percent.LOC', 'percent.HB', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.3', 'seurat_clusters', 'celltype_l1', 'celltype_l2', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'Total_B', 'Total_T_NK', 'Total_Mono', 'cd4_tn2', 'cd8_tn2', 'Treg'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [16]:
# cell2_d=pd.DataFrame(cell_type2.most_common(), columns=['ct1','cnt'])
# ax=cell2_d.plot.bar(x='ct1',y='cnt',rot=90, legend=False)
# ax.bar_label(ax.containers[0])
# ax.set_ylabel('Cell Count')
# ax.set_xlabel('')
# # plt.axhline(y=1000, color="black", linestyle="--")
# # ax.annotate("1000", xy=(0, 0), xytext=(5.5, 1100),color='black')

# cnt_thre=800
# cell2_d_cnt_thre = cell2_d[cell2_d['cnt']>cnt_thre]
# ct_thre = cell2_d_cnt_thre['ct1'].values
# print(ct_thre, len(ct_thre))


In [17]:

adata.write_h5ad(r'E:\pbmc_pe\data\Matrix\win_sc_data_preprocessed.h5ad')

# BUILD model

In [18]:
adata = sc.read_h5ad(r'E:\pbmc_pe\data\Matrix\win_sc_data_preprocessed.h5ad')

In [20]:
for typess in ['Total_Mono', 'cd4_tn2', 'cd8_tn2', 'Treg']:
    # print(typess, ' length:', len(final_cell_types[typess].index))
    print(adata[adata.obs[typess]].shape)

(870, 2000)
(1571, 2000)
(2477, 2000)
(837, 2000)


In [20]:
gene_exclu={}
gene_exclu['Total_B']=["GNLY", "NKG7", "IL7R", "PRF1", "KLRB1", "KLRD1","TMSB4XP4", "Z74021.1", 
                       "IL32", "TRBC1","MTND1P23","GZMA","GZMB", "GZMH","TRDC"]
gene_exclu['Total_T_NK']=["IGHA1","TMSB4XP4", "IGHM","MTND1P23","Z74021.1","JCHAIN","MS4A1","IGLL5", "IGLC2",
                         "IGHG4","OTUD6B-AS1", "IGLC1", "IGHG3", "PPBP", "GABPB1-AS1","PF4", "IGHG2",
                         "CD79A","CD79B","MZB1","IGHD","IGHGP", "CD19"]
gene_exclu['Total_Mono']=["IGHA1","PRF1","GNLY","NKG7","GZMB", "TRBC1","KLRD1", "IRF1", "IL7R"]
gene_exclu['cd4_tn2']=["NKG7", "IGHA1",  "PRF1", "KLRD1", "GNLY","TMSB4XP4", "Z74021.1", "GZMB",  "GZMA"]
gene_exclu['cd8_tn2']=["IGHA1", "TMSB4XP4", "Z74021.1", "RNU4-2"]
gene_exclu['Treg']=["GNLY", "TMSB4XP4","GZMB", "NKG7", "GZMA"]


In [21]:

results=dict()
# for index, row in cell2_d_cnt_thre.iterrows():
for k,v in final_cell_types.items():
    print(k)
    ires=dict()
    cell_type = k
    cell_ids=final_cell_types[k].index
    
    pd_X_train_norm, pd_X_test_norm,y_train,y_test, pat_train,pat_test,scal_mean_var = helper.get_data2(adata, cell_ids, n_cell_pseudo=5)
    
    feat_sel_list = helper.feat_sel_comb(pd_X_train_norm, y_train, percentile=20)
#     feat_dict[i_cell_type]=feat_sel_list
#     feat_f=[x for x in feat_sel[i_cell_type] if x not in gene_excl]
    feat_f=[x for x in feat_sel_list if x not in gene_excl]
    
    feat_f=[x for x in feat_f if x not in gene_exclu[k]]
    
    pd_X_train_sel=pd_X_train_norm[feat_f]
    pd_X_test_sel=pd_X_test_norm[feat_f]
    
    ires['pd_X_tr']=pd_X_train_sel
    ires['pd_X_te']=pd_X_test_sel
    ires['y_tr']=y_train
    ires['y_te']=y_test
    ires['pat_tr']=pat_train
    ires['pat_te']=pat_test
    
    ires['feat_list']=feat_f
    ires['feat_mean']=scal_mean_var
    
    rf_random = helper.cv_train_rf(pd_X_train_sel, y_train)
    rf_cv_results = pd.DataFrame(rf_random.cv_results_)
    print('The best score: ',rf_random.best_score_)
    best_params = rf_random.best_params_
    print('The best parameters: ', best_params)
    
    ires['train_res']=rf_cv_results
    ires['best_param']=best_params
    
    best_model = rf_random.best_estimator_
    pred_prob_y = best_model.predict_proba(pd_X_test_sel)
    pred_y = best_model.predict(pd_X_test_sel)
    ires['best_model']=best_model
    ires['test_res']=pred_prob_y[:,1]
    
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(pd_X_train_sel) # same to x
    ires['shap_values']=shap_values
    
    results[cell_type]=ires


Total_B
The best score:  0.9106498804486153
The best parameters:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 18}
Total_T_NK
The best score:  0.9263566085613313
The best parameters:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 18}
Total_Mono
The best score:  0.9103351629246366
The best parameters:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 9}
cd4_tn2
The best score:  0.9294793126334653
The best parameters:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 9}
cd8_tn2
The best score:  0.9323527869169782
The best parameters:  {'n_estimators': 52, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 14}
Treg
The best score:  0.8434264580629
The best parameters:  {'n_estimators': 126, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 11}


In [22]:
import pickle
with open('pseudo5-final-results.pkl','wb') as f:
    pickle.dump(results, f)

In [24]:
# with open('results.pkl','rb') as f:
#     reltss = pickle.load(f)

In [23]:
import pickle
with open('pseudo5-final-results.pkl','rb') as f:
    res = pickle.load(f)

In [24]:
# 
for typess in ['Total_Mono', 'cd4_tn2', 'cd8_tn2','Treg']:
    i_pat_tr = res[typess]['pat_tr']
    i_pat_te = res[typess]['pat_te']
    
    i_y_tr = res[typess]['y_tr']
    i_y_te = res[typess]['y_te']
    
    sc_cnts_tr = sum([int(x.split('_')[-1]) for x in i_pat_tr.values])
    sc_cnts_te = sum([int(x.split('_')[-1]) for x in i_pat_te.values])
    
    print(typess)
    print(sc_cnts_tr, len(i_pat_tr), np.sum(i_y_tr==1),np.sum(i_y_tr==0)) 
    print(sc_cnts_te, len(i_pat_te), np.sum(i_y_te==1),np.sum(i_y_te==0) )
    
    
    

Total_Mono
608 126 39 87
262 55 17 38
cd4_tn2
1097 226 47 179
474 98 20 78
cd8_tn2
1731 354 80 274
746 152 34 118
Treg
590 123 33 90
247 54 14 40


590